In [1]:
# coding:utf-8

from __future__ import division
from __future__ import print_function

import datetime
import logging

import numpy as np

import data_helpers
import model_all_stacked

Using Theano backend.


### Настройка логгера

In [2]:
logging.basicConfig(filename='log\all_results.log',
                    format='[%(asctime)s] [%(levelname)s] %(message)s',
                    level=logging.DEBUG)

lg = logging.getLogger("L")
lg.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(message)s")
ch.setFormatter(formatter)
lg.addHandler(ch)

### Установка параметров

In [3]:
np.random.seed(0123)  # for reproducibility

# set parameters:

# save = False
# model_name_path = 'params/model.json'
# model_weights_path = 'params/model_weights.h5'

train_file = "data/train.csv"
test_file = "data/test.csv"

latent_dimension = 100 #125
learning_rate = 0.01
maxlen = 10 #25

batch_size = 100
test_batch_size = 20
nb_epoch = 50

SHOW_STEP = 20
NUM = 2 # number of model

## Загрузка модели

In [4]:
lg.info('Loading data...')
vocab, reverse_vocab, vocab_size, check = data_helpers.create_vocab_set()

if NUM in (0, 1, 2):
    mini_batch_generator = data_helpers.mini_batch_generator
else:
    raise NotImplemented()

lg.info('Build model...')
if NUM == 0:
    model = model_all_stacked.construct_model(maxlen, vocab_size * 3, vocab_size, latent_dimension, lr=learning_rate)
elif NUM == 1:
    model = model_all_stacked.construct_simpliest_model(maxlen, vocab_size * 3, vocab_size, latent_dimension)
elif NUM == 2:
    model = model_all_stacked.construct_multilayer_model(maxlen, vocab_size * 3, vocab_size, latent_dimension,
                                                         lr=learning_rate,
                                                         clipnorm=10.0)


2016-10-31 17:52:21,806 [INFO] Loading data...
2016-10-31 17:52:21,811 [INFO] Build model...


In [5]:
lg.info('Fit model...')
initial = datetime.datetime.now()

for e in xrange(nb_epoch):
    
    batches = mini_batch_generator(train_file,
                                   vocab, vocab_size, check, maxlen, 
                                   batch_size=batch_size)

    test_batches = mini_batch_generator(test_file,
                                        vocab, vocab_size, check, maxlen, 
                                        batch_size=test_batch_size)

    loss = 0.0
    step = 1
    start = datetime.datetime.now()

    lg.info('-------- epoch {} --------'.format(e))

    for x_train, y_train, x_text_tr, y_text_tr in batches:

        f = model.train_on_batch(x_train, y_train)
        loss += f
        loss_avg = loss / step

        if step % SHOW_STEP == 0:
            lg.info('- TRAINING step {} \t loss {}'.format(step, loss_avg))
        step += 1

    test_loss = 0.0
    test_step = 0

    lg.info(" -- TESTING NOW -- ")

    for x_test_batch, y_test_batch, x_text, y_text in test_batches:
        f_ev = model.test_on_batch(x_test_batch, y_test_batch)
        test_loss += f_ev
        test_loss_avg = test_loss / test_step
        test_step += 1
        
        if test_step % SHOW_STEP == 0:
            predicted_seq = model.predict(np.array([x_test_batch[0]]))
            if False:
                lg.info('- TESTING step {}\tloss {}'.format(test_step, test_loss_avg))
                lg.info(
                    'Shapes x {} y_true {} y_pred {}'.format(
                        x_test_batch[0].shape,
                        y_test_batch[0].shape,
                        predicted_seq[0].shape))
                lg.info(u'Input:       \t[' + "|".join(map(lambda x:x[:maxlen], list(x_text[0]))) + "] -> ? ")
                lg.info(u'Expected:    \t[' + y_text[0] + "]")
                lg.info(u'Predicted: \t[' + data_helpers.decode_data(predicted_seq, reverse_vocab) + "]")
                lg.info('----------------------------------------------------------------')
            else:
                lg.info(u'- TESTING step {}\tloss {}. [{}] ~=[{}]'.format(test_step,
                                                                         test_loss_avg,
                                                                         y_text[0],
                                                                         data_helpers.decode_data(predicted_seq, reverse_vocab)))

    stop = datetime.datetime.now()
    e_elap = stop - start
    t_elap = stop - initial
    lg.info('Epoch {}. Loss: {}\nEpoch time: {}. Total time: {}\n'.format(e, test_loss, e_elap, t_elap))

    # if save:
    #     print('Saving model params...')
    #     json_string = model.to_json()
    #     with open(model_name_path, 'w') as f:
    #         json.dump(json_string, f)
    #
    #     model.save_weights(model_weights_path)

2016-10-31 17:52:26,799 [INFO] Fit model...
2016-10-31 17:52:26,803 [INFO] -------- epoch 0 --------
2016-10-31 17:55:04,594 [INFO] - TRAINING step 20 	 loss 7.27880854607
2016-10-31 17:55:15,799 [INFO] - TRAINING step 40 	 loss 7.89276953936
2016-10-31 17:55:26,354 [INFO] - TRAINING step 60 	 loss 8.13149441878
2016-10-31 17:55:40,922 [INFO] - TRAINING step 80 	 loss 8.57136520743
2016-10-31 17:55:49,392 [INFO] - TRAINING step 100 	 loss 8.69704077244
2016-10-31 17:55:57,972 [INFO] - TRAINING step 120 	 loss 8.94475664695
2016-10-31 17:56:06,241 [INFO] - TRAINING step 140 	 loss 9.09364807606
2016-10-31 17:56:07,852 [INFO]  -- TESTING NOW -- 
2016-10-31 17:56:31,701 [INFO] - TESTING step 20	loss 7.85150470232. [антраша] ~=[__________]
2016-10-31 17:56:32,459 [INFO] - TESTING step 40	loss 8.72500029588. [бытность] ~=[__________]
2016-10-31 17:56:33,202 [INFO] - TESTING step 60	loss 8.82245196326. [примеры] ~=[__________]
2016-10-31 17:56:33,974 [INFO] - TESTING step 80	loss 9.306342450

KeyboardInterrupt: 